# Lab3

---

## Task

- Реализовать метод решения СЛАУ, на выбор: метод вращений или метод отражений.
- Вычислить числа обусловленности.
- Протестировать на тех же матрицах, что использовались в задании 2; сравнить.

# Solution

---

In [ ]:
import numpy as np
from scipy import linalg as la

from utils.lineq import *

def rotation_method_solve(matr, b):
    """, 
    A: matr
    We solve the equation Ax = b, where A = QR
    """
    Q, R = qr_decompose(matr)

    new_b = Q.T.dot(b)
    x = lu_solve(R, new_b)

    return x

def create_rotation_matrix(n, sin, cos, i, j):
    matr = np.identity(n)
    matr[i, i] = cos
    matr[j, j] = cos
    matr[i, j] = -sin
    matr[j, i] = sin
    
    return matr

def qr_decompose(A: np.array):
    n = A.shape[0]
    Q = np.identity(n)
    R = A.copy()
    
    for j in range(n):
        for i in range(n - 1, j, -1):
            sin, cos = get_sin_cos(R[i - 1, j], R[i, j])
            rotation_matrix = create_rotation_matrix(n, sin, cos, i, j)
            Q = Q @ rotation_matrix
            R = rotation_matrix.T @ R
    return Q, R

## Experimental research

In [ ]:
from utils.cond_nums import *
from utils.matrices import *

def print_cond_nums_for_rotation_method(A):
    Q, R = qr_decompose(A)
    for name, matr in (("A", A), ("Q", Q), ("R", R)):
        spec_cn, bulk_cn, ang_cn = compute_matr_condition_nums(matr)
        print(f"Matrix {name}:")
        print(f"    Spectral condition number : {spec_cn}")
        print(f"    Bulk condition number     : {bulk_cn}")
        print(f"    Angular condition number  : {ang_cn}")



In [ ]:
A = create_random_matrix(rank)
rank = 10
print_cond_nums_for_rotation_method(A)

x = np.ones(A.shape[0])
b = A @ x

actual = rotation_method_solve(A, b)
expected = la.solve(A, b)

print(f"\nError: {la.norm(actual - x)}")
print(f"\nError: {la.norm(expected - x)}")

In [ ]:
A = create_tridiagonal_matrix(rank)
print_cond_nums_for_rotation_method(A)

b = np.ones(A.shape[0])
actual = rotation_method_solve(A, b)
expected = np.linalg.solve(A, b)

print(f"\nError: {la.norm(actual - expected)}")

In [ ]:
rank = 5
A = create_tridiagonal_matrix(rank)
print_cond_nums_for_rotation_method(A)

b = np.ones(A.shape[0])

actual = rotation_method_solve(A, b)
expected = lu_solve(A, b)

print(f"\nError: {la.norm(actual - expected)}")

In [ ]:
rank = 10
A = create_hilbert_matrix(rank)
print_cond_nums_for_rotation_method(A)

b = np.ones(A.shape[0])
actual = rotation_method_solve(A, b)
expected = A.dot(actual)

print(f"\nError: {la.norm(actual - expected)}")